In [2]:
# Import Packages
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import numpy as np
import pickle
import tensorflow as tf
import threading
import time



# Load Model & Metadata
MODEL_PATH = r"C:\Users\ACER\Desktop\Projects\Potato Leaf Disease Classification using Deep Learning Custom CNN Mini Project\potato_model.h5"
META_PATH = r"C:\Users\ACER\Desktop\Projects\Potato Leaf Disease Classification using Deep Learning Custom CNN Mini Project\metadata.pkl"

model = tf.keras.models.load_model(MODEL_PATH)

with open(META_PATH,"rb") as f:
    metadata = pickle.load(f)

class_names = metadata["class_names"]
img_size = metadata["image_size"]

if isinstance(img_size, int):
    img_size = (img_size, img_size)




# Global
current_arr = None
theme = "light"
loading = False



# Themes
themes = {
    "light": {
        "bg":"#e9e9e9",
        "fg":"black",
        "accent":"#006600",
        "result":"#0033aa"
    },
    "dark": {
        "bg":"#1e1e1e",
        "fg":"white",
        "accent":"#00ff99",
        "result":"#66aaff"
    }
}


# Preprocess
def preprocess(path):
    img = Image.open(path).convert("RGB")
    img = img.resize(img_size)
    arr = np.array(img)/255.0
    return np.expand_dims(arr,0)




# Display Image
def display(img):
    img.thumbnail((420,420))
    tk_img = ImageTk.PhotoImage(img)
    panel.config(image=tk_img)
    panel.image = tk_img




# Upload
def upload():
    global current_arr
    path = filedialog.askopenfilename()
    if not path:
        return

    img = Image.open(path).convert("RGB")
    display(img)

    current_arr = preprocess(path)

    result_label.config(text="Image Uploaded.\nPlease Press Predict.", fg=themes[theme]["accent"])




# Loading Animation
def spin():
    anim = ["|","/","—","\\"]
    i = 0
    while loading:
        result_label.config(text="Predicting " + anim[i%4])
        i += 1
        time.sleep(0.1)




# Predict (Threaded)
def predict():
    global loading

    if current_arr is None:
        result_label.config(text="Upload an image first!", fg="red")
        return

    loading = True
    threading.Thread(target=spin, daemon=True).start()
    threading.Thread(target=do_predict, daemon=True).start()



def do_predict():
    global loading

    pred = model.predict(current_arr)[0]
    cls = class_names[np.argmax(pred)]
    conf = np.max(pred)*100

    loading = False

    result_label.config(
        text=f"Prediction: {cls}\n\nConfidence: {conf:.2f}%",
        fg=themes[theme]["result"],
        font=("Segoe UI", 14, "bold")
    )




# Toggle Theme
def toggle_theme():
    global theme
    theme = "dark" if theme=="light" else "light"
    apply_theme()



def apply_theme():
    t = themes[theme]
    root.config(bg=t["bg"])
    title.config(bg=t["bg"], fg=t["accent"])
    panel.config(bg=t["bg"])
    result_label.config(bg=t["bg"], fg=t["fg"])
    btn_frame.config(bg=t["bg"])
    btn_dark.config(bg=t["accent"], fg=t["bg"])
    btn_upload.config(bg="#28a745")
    btn_predict.config(bg="#007bff")




# GUI
root = tk.Tk()
root.title("Potato Leaf AI Classifier")
root.geometry("650x750")



# Title
title = tk.Label(root,
                    text="🌱 Potato Leaf Disease Classifier",
                    font=("Segoe UI",20,"bold"))
title.pack(pady=10)



# Image
panel = tk.Label(root)
panel.pack(pady=10)



# Result
result_label = tk.Label(root,
                        text="Please upload an Image",
                        font=("Segoe UI",14),
                        justify="center")
result_label.pack(pady=10)



# Buttons
btn_frame = tk.Frame(root)
btn_frame.pack(pady=10)

btn_upload = tk.Button(btn_frame,
                        text="Upload Image",
                        font=("Segoe UI",13),
                        command=upload,
                        width=15)
btn_upload.grid(row=0, column=0, padx=10)

btn_predict = tk.Button(btn_frame,
                        text="Predict",
                        font=("Segoe UI",13),
                        command=predict,
                        width=15)
btn_predict.grid(row=0, column=1, padx=10)



# Dark Mode Switch
btn_dark = tk.Button(root,
                        text="Toggle Theme",
                        font=("Segoe UI",12),
                        command=toggle_theme)
btn_dark.pack(pady=10)



# Apply initial theme
apply_theme()



root.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 305ms/step
